In [1]:
import sys
import lzma
from s2clientprotocol.sc2api_pb2 import Response, ResponseObservation
from MapAnalyzer.utils import import_bot_instance
from MapAnalyzer import MapData
import pickle
import numpy as np
import matplotlib.pyplot as plt
sys.path.append("../src")
sys.path.append("../src/ares")
from sc2.position import Point2
%load_ext line_profiler
%load_ext Cython

C:\Users\Tom\AppData\Local\pypoetry\Cache\virtualenvs\ares-sc2-S437wHWG-py3.9\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BERLINGRAD = "../tests/pickle_data/BerlingradAIE.xz"
with lzma.open(BERLINGRAD, "rb") as f:
    raw_game_data, raw_game_info, raw_observation = pickle.load(f)
bot = import_bot_instance(raw_game_data, raw_game_info, raw_observation)
data = MapData(bot)

2023-05-23 10:48:26.160 | INFO     | MapAnalyzer.MapData:__init__:122 - dev Compiling Berlingrad AIE 
 Version dev Map Compilation Progress : 0.4it [00:00,  1.49it/s]


# Converting `is_position_safe` to cython

In [69]:
grid = data.get_pyastar_grid()
position = bot.enemy_start_locations[0]

## Python version of `is_position_safe`

In [70]:
def is_position_safe(
    grid: np.ndarray,
    position: Point2,
    weight_safety_limit: float = 1.0,
) -> bool:
    weight: float = grid[position.rounded]
    # np.inf check if drone is pathing near a spore crawler
    return weight == np.inf or weight <= weight_safety_limit

In [67]:
%timeit is_position_safe(grid, position)

4.67 µs ± 76.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [71]:
%lprun -f is_position_safe is_position_safe(grid, position)

Timer unit: 1e-07 s

Total time: 5.35e-05 s
File: C:\Users\Tom\AppData\Local\Temp\ipykernel_24904\2777119042.py
Function: is_position_safe at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def is_position_safe(
     2                                               grid: np.ndarray,
     3                                               position: Point2,
     4                                               weight_safety_limit: float = 1.0,
     5                                           ) -> bool:
     6         1        157.0    157.0     29.3      weight: float = grid[position.rounded]
     7                                               # np.inf check if drone is pathing near a spore crawler
     8         1        378.0    378.0     70.7      return weight == np.inf or weight <= weight_safety_limit

## Cython version of `is_position_safe`

In [73]:
%%cython
import numpy as np
cimport numpy as cnp
from cython cimport boundscheck, wraparound
@boundscheck(False)
@wraparound(False)
cpdef bint is_position_safe(
    cnp.ndarray[cnp.npy_float32, ndim=2] grid,
    (unsigned int, unsigned int) position,
    double weight_safety_limit = 1.0,
):
    cdef double weight = 0.0
    weight = grid[position[0], position[1]]
    # np.inf check if drone is pathing near a spore crawler
    return weight == np.inf or weight <= weight_safety_limit

In [74]:
%timeit is_position_safe(grid, position.rounded)

812 ns ± 8.41 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


# Alternative to `python-sc2`'s `units.closest_to`

In [78]:
units = bot.all_units
position = bot.enemy_start_locations[0]
unit = units[0]

In [79]:
# slower using closest_to(Point2)
%timeit units.closest_to(position)

301 µs ± 4.34 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [109]:
# this is faster since distance between all units is cached
%timeit units.closest_to(unit)

115 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [139]:
%%cython
from cython cimport boundscheck, wraparound

cdef double euclidean_distance_squared(
        (float, float) p1,
        (float, float) p2
):
    return (p1[0] - p2[0]) ** 2 + (p1[1] - p2[1]) ** 2

@boundscheck(False)
@wraparound(False)
cpdef object closest_to((float, float) position, object units):
    cdef:
        object closest = units[0]
        double closest_dist = 999.9
        double dist = 0.0
        unsigned int len_units = len(units)
        (float, float) pos
        
    for i in range(len_units):
        unit = units[i]
        pos = unit.position
        dist = euclidean_distance_squared((pos[0], pos[1]), (position[0], position[1]))
        if dist < closest_dist:
            closest_dist = dist
            closest = unit
            
    return closest


In [140]:
%timeit closest_to(position, units)

14.9 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
